### query enhancement

In [13]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableMap

In [2]:
# doc loader
loader=TextLoader("langchain_crewai_dataset.txt")
docs=loader.load()

# text splitting

splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
chunks=splitter.split_documents(docs)


In [3]:
embeddings=OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore=FAISS.from_documents(chunks,embeddings)


In [4]:
# Retriever
retriever=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":3})

In [5]:
retriever.invoke("langchain")

[Document(id='de313744-9b84-4453-9c78-fb869657ac56', metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use LangChain to create end-to-end pipelines that connect LLMs with tools, APIs, vector databases, and other knowledge sources. (v8)'),
 Document(id='3715d2e7-7d34-4843-85a2-10a86b660f9f', metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='At the heart of LangChain lies the concept of chains, which are sequences of calls to LLMs and other tools. Chains can be simple, such as a single prompt fed to an LLM, or complex, involving multiple conditionally executed steps. LangChain makes it easy to compose and reuse chains using standard patterns like Stuff, Map-Reduce

In [17]:
# llm
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
llm=init_chat_model("openai:o4-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001E73F15CE10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E73F15E850>, root_client=<openai.OpenAI object at 0x000001E73F15E490>, root_async_client=<openai.AsyncOpenAI object at 0x000001E73F15E5D0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [18]:
# QUERY Enhancement
query_expansion_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.

Original query: "{query}"

Expanded query:
""")


In [19]:
query_expansion_chain=query_expansion_prompt | llm | StrOutputParser()

In [20]:
query_expansion_chain.invoke({"query":"langchain memory"})

'Expanded query:\n\n“langchain memory” OR “LangChain Memory Module” OR “LangChain State Management” OR “conversation memory in LangChain” OR “LangChain context persistence” OR “chatbot session memory” OR “conversation history storage” OR “context window management” OR “memory retriever” OR “retrieval-augmented generation (RAG)” OR “vector-store memory” OR “embedding store” OR “persistent memory (Chroma, Pinecone, Redis)” OR “in-memory cache” OR “short-term vs. long-term memory retention” OR “LangChain Memory API” OR “contextual memory management”'

In [21]:
# RAG answering prompt
answer_prompt=PromptTemplate.from_template(
    """
answer the question given only the context below 
context:
{context}

question:{input}
    """
)
answer_prompt

PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nanswer the question given only the context below \ncontext:\n{context}\n\nquestion:{input}\n    ')

In [22]:
document_chain=create_stuff_documents_chain(llm,prompt=answer_prompt)
rag_chain=(
    RunnableMap(
        {
            "input": lambda x:x['input'],
            "context": lambda x:retriever.invoke( (query_expansion_chain.invoke({"query":x['input']})))
        }
    )
    | document_chain
)
rag_chain

{
  input: RunnableLambda(...),
  context: RunnableLambda(...)
}
| RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableLambda(format_docs)
  }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
  | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nanswer the question given only the context below \ncontext:\n{context}\n\nquestion:{input}\n    ')
  | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001E73F15CE10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E73F15E850>, root_client=<openai.OpenAI object at 0x000001E73F15E490>, root_async_client=<openai.AsyncOpenAI object at 0x000001E73F15E5D0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
  | StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=

In [23]:
# Step 6: Run query
query = {"input": "What types of memory does LangChain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_chain.invoke(query)
print("✅ Answer:\n", response)

Expanded query:  
“What types of memory (also called memory modules, context-retention layers or stateful storage) does the LangChain Python SDK support for conversational agents and retrieval-augmented generation? For example:  
- Short-term/ephemeral memory such as ConversationBufferMemory, TokenBufferMemory, Window (sliding-window) memory  
- Long-term/persistent memory such as ConversationSummaryMemory, EntityMemory or KnowledgeGraphMemory  
- Vector-store memory backends (e.g. FAISS, Pinecone, Weaviate), Redis/SQL/file-based memory stores  
- Memory abstractions and interfaces (Memory class, memory retriever, callback manager)  
Key synonyms and related terms: memory management, context window, session persistence, chat history storage, context caching, state management, memory adapters, memory plugins—within the broader LangChain framework for RAG, agents, and LLM-driven applications.”
✅ Answer:
 LangChain currently provides memory modules such as:  
• ConversationBufferMemory – 

In [24]:
# Step 6: Run query
query = {"input": "CrewAI agents?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_chain.invoke(query)
print("\n✅ Answer:\n", response)

Here’s one possible expanded query that adds synonyms, related technical terms and useful context for better recall:

("CrewAI agents"  
 OR "Crew.ai autonomous agents"  
 OR "AI‐driven crew management agents"  
 OR "intelligent crew scheduling agents"  
 OR "autonomous workforce planning agents"  
 OR "digital crew assistants"  
 OR "virtual crew planners"  
 OR "multi‐agent crewing system")  
AND  
("crew management"  
 OR "crew scheduling"  
 OR "roster planning"  
 OR "shift allocation"  
 OR "resource optimization"  
 OR "workforce optimization")  
AND  
("machine learning"  
 OR "reinforcement learning"  
 OR "multi‐agent systems"  
 OR "decision support systems"  
 OR "optimization algorithms"  
 OR "conversational AI"  
 OR "digital assistants")

✅ Answer:
 CrewAI agents are autonomous, LLM-powered “crew members” each defined by:  
1. A clear purpose and goal  
2. A role within the crew (e.g. researcher, planner, executor)  
3. A set of tools or APIs they’re allowed to invoke  